In [1]:
import csv
from pycocotools.coco import COCO
import numpy as np
from skimage import io
from glob import glob
import os
from tqdm import tqdm

In [2]:
def get_cats():
    coco = COCO("HotelsCocos/hotels.json")
    cats = coco.loadCats(coco.getCatIds())
    return cats

In [3]:
cats = get_cats()
hotels_labels = []
for idx, cat in enumerate(cats):
    hex = cat['color'].lstrip('#')
    rgb = list(tuple(int(hex[i:i+2], 16) for i in (0, 2, 4)))
    hotels_labels.append(str(cat['name']))

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [4]:
ade_labels = []

with open('data/object150_info.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for idx, row in enumerate(reader):
        ade_labels.append(row[-1])

In [9]:
map = {}
for h in hotels_labels:
    search_term = h.replace(" ", "").split("/")
    for a in ade_labels:
        for s in search_term:
            if s in a:
                # print(s, a)
                map[hotels_labels.index(h)+1] = ade_labels.index(a)+1, "hotel:" + h,"ade: " + a
        if hotels_labels.index(h)+1 not in map:
                map[hotels_labels.index(h)+1] = 0,"hotel:" + h,"ade: " + a


In [11]:
test = [f for f in map.keys() if map[f][0]==0]
len(test)

11

In [6]:
imgs = [f for f in glob("/pless_nfs/home/mdt_/semantic-segmentation-pytorch/imgs/HotelsData/*.png")]
    
for img in tqdm(imgs, desc='Adjusting Hotels'):
    op = io.imread(img)
    for nc in map.keys():
        op[op == nc] = map[nc]
    io.imsave(os.path.join("imgs/Hotels150", os.path.basename(img)), op)

Adjusting Hotels:  46%|████▋     | 124/267 [00:03<00:03, 38.05it/s]


KeyboardInterrupt: 

In [33]:
from scipy.io import loadmat

ade_colors = loadmat('data/color150.mat')['colors']

In [34]:
l = [[i*10, i*10, i*10] for i in range(12)]
ade_colors = ade_colors.tolist()
for i in range(len(l)):
    ade_colors.append(l[i])
# l = np.concatenate(ade_colors, l)

import pickle

with open("Colors/colors_Hotels_and_ADE_162.pckl", "wb") as pckl:
    pickle.dump(ade_colors, pckl)